# Customer Churn Prediction - EDA and Model Development
This notebook analyzes user behavior data from a music streaming service to predict customer churn. The analysis includes data cleaning, feature engineering, and model development.


In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

## 1. Data Loading and Initial Exploration


In [3]:
dataframe = pd.read_json("data/customer_churn_mini.json", lines=True)

print(f"Dataset shape: {dataframe.shape}")
print("First few rows:")
dataframe.head()

Dataset shape: (286500, 18)
First few rows:


,ts,userId,sessionId,page,auth,method,status,level,itemInSession,location,userAgent,lastName,firstName,registration,gender,artist,song,length
0,1538352117000,30,29,NextSong,Logged In,PUT,200,paid,50,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Martha Tilston,Rockpools,277.89016
1,1538352180000,9,8,NextSong,Logged In,PUT,200,free,79,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,Five Iron Frenzy,Canada,236.09424
2,1538352394000,30,29,NextSong,Logged In,PUT,200,paid,51,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Adam Lambert,Time For Miracles,282.82730
3,1538352416000,9,8,NextSong,Logged In,PUT,200,free,80,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,Enigma,Knocking On Forbidden Doors,262.71302
4,1538352676000,30,29,NextSong,Logged In,PUT,200,paid,52,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Daft Punk,Harder Better Faster Stronger,223.60771


In [4]:
# Check data types and missing values
dataframe.info()

# Check missing values summary
missing_summary = dataframe.isna().sum().sort_values(ascending=False)
missing_pct = (missing_summary / dataframe.shape[0]).round(4)

print("Missing values by column:")
for col, count in missing_summary.items():
    if count > 0:
        print(f"{col}: {count:,} ({missing_pct[col]:.1%})")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286500 entries, 0 to 286499
Data columns (total 18 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ts             286500 non-null  int64  
 1   userId         286500 non-null  object 
 2   sessionId      286500 non-null  int64  
 3   page           286500 non-null  object 
 4   auth           286500 non-null  object 
 5   method         286500 non-null  object 
 6   status         286500 non-null  int64  
 7   level          286500 non-null  object 
 8   itemInSession  286500 non-null  int64  
 9   location       278154 non-null  object 
 10  userAgent      278154 non-null  object 
 11  lastName       278154 non-null  object 
 12  firstName      278154 non-null  object 
 13  registration   278154 non-null  float64
 14  gender         278154 non-null  object 
 15  artist         228108 non-null  object 
 16  song           228108 non-null  object 
 17  length         228108 non-nul

## 2. Data Cleaning and Preprocessing
### Timestamp Conversion


In [5]:
dataframe['ts'] = pd.to_datetime(dataframe['ts'], unit='ms')
dataframe['registration'] = pd.to_datetime(dataframe['registration'], unit='ms')

print(f"Date range: {dataframe['ts'].min()} to {dataframe['ts'].max()}")

Date range: 2018-10-01 00:01:57 to 2018-12-03 01:11:16


In [6]:
# Verify data integrity for music-related features
# Songs should only have missing values when page != 'NextSong'
nextsong_pages = dataframe.query("page == 'NextSong'")
missing_song_data = nextsong_pages[['length', 'song', 'artist']].isna().any(axis=1).sum()
print(f"NextSong pages with missing music data: {missing_song_data}")

NextSong pages with missing music data: 0


### User ID Data Quality Investigation

The dataset contains empty string userIds for logged-out sessions. We need to handle these cases and potentially recover user information where possible.

In [7]:
# Sort data chronologically for proper session analysis
dataframe = dataframe.sort_values(['ts', 'sessionId', 'itemInSession']).reset_index(drop=True)

# Investigate empty userIds
non_numeric_users = dataframe[pd.to_numeric(dataframe['userId'], errors='coerce').isna()]['userId'].unique()
print(f"Non-numeric userIds found: {non_numeric_users}")

# Convert empty strings to NaN and make userId numeric
dataframe['userId'] = pd.to_numeric(dataframe['userId'], errors='coerce')
print(f"Missing userIDs after conversion: {dataframe['userId'].isna().sum():,}")

# Analyze patterns in missing userIds
print("Auth status for missing userIds:")
print(dataframe[dataframe['userId'].isna()]['auth'].value_counts())

print("Pages visited by users with missing userIds:")
dataframe[dataframe['userId'].isna()]['page'].value_counts().head(10)

Non-numeric userIds found: ['']
Missing userIDs after conversion: 8,346
Auth status for missing userIds:
auth
Logged Out    8249
Guest           97
Name: count, dtype: int64
Pages visited by users with missing userIds:


page
Home                   4375
Login                  3241
About                   429
Help                    272
Register                 18
Error                     6
Submit Registration       5
Name: count, dtype: int64

### Session Analysis

Sessions are reused across users. We can use sessionId with itemInSession to track users even after logout, as itemInSession always increments by 1 for every user action.

In [8]:
# Check for multi-user sessions
not_null_users = dataframe[dataframe['userId'].notna()]
session_user_counts = not_null_users.groupby('sessionId')['userId'].nunique()
multi_user_sessions = session_user_counts[session_user_counts > 1]

print(f"Sessions with multiple users: {len(multi_user_sessions):,}")
print("This confirms that sessions are reused across different users")

Sessions with multiple users: 466
This confirms that sessions are reused across different users


### User ID Imputation Strategy

Since itemInSession increments sequentially within a session, we can use this pattern to impute missing userIds by:
1. Looking at adjacent records in the same session
2. Finding the user with the most consistent itemInSession sequence
3. Using timestamp proximity as a tiebreaker

In [9]:
def is_valid_sequence_assignment(user_items_array, item):
    """Check if assigning an item to a user creates a valid consecutive sequence."""
    if len(user_items_array) == 0:
        return True
    
    min_item = user_items_array[0]
    max_item = user_items_array[-1]
    
    if item == max_item + 1 or item == min_item - 1:
        return True
    elif min_item < item < max_item:
        for existing_item in user_items_array:
            if abs(existing_item - item) == 1:
                return True
        return False
    else:
        return False

def fill_missing_userids(df):
    """Fill missing userIDs using session sequence logic."""
    df = df.copy()
    df['imputed'] = False
    df['ts'] = pd.to_datetime(df['ts'])
    
    userId_col = df.columns.get_loc('userId')
    imputed_col = df.columns.get_loc('imputed')

    for session in df[df['userId'].isna()]['sessionId'].unique():
        mask = df['sessionId'] == session
        session_df = df[mask].copy()
        
        userId_array = session_df['userId'].values
        itemInSession_array = session_df['itemInSession'].values
        ts_array = pd.to_datetime(session_df['ts']).values
        
        max_iterations = len(session_df)
        iteration = 0
        
        while iteration < max_iterations:
            iteration += 1
            filled_any = False
            user_items_cache = {}
            
            missing_mask = pd.isna(userId_array)
            missing_indices = np.where(missing_mask)[0]
            
            if len(missing_indices) == 0:
                break
            
            for i in missing_indices:
                missing_ts = ts_array[i]
                missing_item = itemInSession_array[i]
                
                forward_user = None
                backward_user = None
                forward_ts = None
                backward_ts = None
                
                # Look forward
                for j in range(i + 1, len(userId_array)):
                    candidate_user = userId_array[j]
                    if not pd.isna(candidate_user):
                        if candidate_user not in user_items_cache:
                            user_mask = userId_array == candidate_user
                            user_items_cache[candidate_user] = np.sort(itemInSession_array[user_mask])
                        
                        user_items = user_items_cache[candidate_user]
                        
                        if missing_item not in user_items:
                            if is_valid_sequence_assignment(user_items, missing_item):
                                forward_user = candidate_user
                                forward_ts = ts_array[j]
                                break
                
                # Look backward
                for j in range(i - 1, -1, -1):
                    candidate_user = userId_array[j]
                    if not pd.isna(candidate_user):
                        if candidate_user not in user_items_cache:
                            user_mask = userId_array == candidate_user
                            user_items_cache[candidate_user] = np.sort(itemInSession_array[user_mask])
                        
                        user_items = user_items_cache[candidate_user]
                        
                        if missing_item not in user_items:
                            if is_valid_sequence_assignment(user_items, missing_item):
                                backward_user = candidate_user
                                backward_ts = ts_array[j]
                                break
                
                # Choose user based on timestamp proximity
                chosen_user = None
                
                if forward_user is not None and backward_user is not None:
                    forward_diff = abs((forward_ts - missing_ts).astype('timedelta64[s]').astype(int))
                    backward_diff = abs((backward_ts - missing_ts).astype('timedelta64[s]').astype(int))
                    
                    if forward_diff <= backward_diff:
                        chosen_user = forward_user
                    else:
                        chosen_user = backward_user
                elif forward_user is not None:
                    chosen_user = forward_user
                elif backward_user is not None:
                    chosen_user = backward_user
                
                if chosen_user is not None:
                    userId_array[i] = chosen_user
                    session_df.iloc[i, imputed_col] = True
                    filled_any = True
                    if chosen_user in user_items_cache:
                        del user_items_cache[chosen_user]
            
            if not filled_any:
                break
        
        session_df.iloc[:, userId_col] = userId_array
        df.loc[mask, 'userId'] = session_df['userId'].values
        df.loc[mask, 'imputed'] = session_df['imputed'].values

    return df

def map_user_attributes(df):
    """Map user attributes based on userId."""
    df = df.copy()
    user_cols = ['location', 'userAgent', 'lastName', 'firstName', 'registration', 'gender']
    
    user_map = df[df['userId'].notna()].groupby('userId')[user_cols].first()
    
    for col in user_cols:
        df[col] = df['userId'].map(user_map[col]).fillna(df[col])
    
    return df

In [10]:
# Apply imputation
dataframe_filled = fill_missing_userids(dataframe)
dataframe_filled = map_user_attributes(dataframe_filled)

print(f"Imputation results:")
print(f"- Records imputed: {dataframe_filled['imputed'].sum():,}")
print(f"- Still missing userIDs: {dataframe_filled['userId'].isna().sum():,}")

# Remove remaining NaN userIds
dataframe_filled = dataframe_filled.dropna(subset=['userId']).reset_index(drop=True)
dataframe_filled['userId'] = dataframe_filled['userId'].astype(int)

# Create location features
dataframe_filled['city'] = dataframe_filled['location'].str.split(',').str[0]
dataframe_filled['state'] = dataframe_filled['location'].str.split(',').str[1].str.strip()

print(f"Final dataset shape: {dataframe_filled.shape}")
print(f"Unique users: {dataframe_filled['userId'].nunique():,}")

Imputation results:
- Records imputed: 8,183
- Still missing userIDs: 163
Final dataset shape: (286337, 21)
Unique users: 225


## 3. Exploratory Data Analysis

### User Demographics

In [11]:
# Create user-level dataset for demographics
user_data = dataframe_filled.groupby('userId').last()

print(f"User demographics summary:")
print(f"- Total unique users: {len(user_data):,}")
print(f"- Gender distribution:\n{user_data['gender'].value_counts()}")
print(f"- Subscription level distribution:\n{user_data['level'].value_counts()}")
print(f"- Paid users percentage: {(user_data['level'] == 'paid').mean():.1%}")

User demographics summary:
- Total unique users: 225
- Gender distribution:
gender
M    121
F    104
Name: count, dtype: int64
- Subscription level distribution:
level
paid    145
free     80
Name: count, dtype: int64
- Paid users percentage: 64.4%


In [51]:
# Geographic distribution
state_counts = user_data['state'].value_counts()
fig = px.bar(x=state_counts.index[:20], y=state_counts.values[:20],
             title='Top 20 States by User Count',
             labels={'x': 'State', 'y': 'Number of Users'})
fig.show()

# Gender distribution
gender_counts = user_data['gender'].value_counts()
fig = px.pie(values=gender_counts.values, names=gender_counts.index,
             title='User Gender Distribution')
fig.show()

# Subscription level by gender
level_gender = user_data.groupby(['level', 'gender']).size().reset_index(name='count')
fig = px.bar(level_gender, x='level', y='count', color='gender',
             title='Subscription Level by Gender',
             barmode='group')
fig.show()

In [50]:
# Page visit patterns
page_counts = dataframe_filled['page'].value_counts()
fig = px.bar(x=page_counts.index[:15], y=page_counts.values[:15],
             title='Page Visit Distribution',
             labels={'x': 'Page', 'y': 'Count'})
fig.show()

# Authentication status
auth_counts = dataframe_filled['auth'].value_counts()
fig = px.bar(x=auth_counts.index, y=auth_counts.values,
             title='Authentication Status Distribution',
             labels={'x': 'Auth Status', 'y': 'Count'})
fig.show()

In [13]:
# Music listening analysis
songs_df = dataframe_filled[dataframe_filled['song'].notna()]
print(f"Total unique songs: {songs_df['song'].nunique():,}")
print(f"Total unique artists: {songs_df['artist'].nunique():,}")
print(f"Average song length: {songs_df['length'].mean()/60:.1f} minutes")

Total unique songs: 58,480
Total unique artists: 17,655
Average song length: 4.2 minutes


In [52]:
# Top artists
top_artists = songs_df['artist'].value_counts().head(20)
fig = px.bar(x=top_artists.values, y=top_artists.index,
             orientation='h',
             title='Top 20 Most Played Artists',
             labels={'x': 'Play Count', 'y': 'Artist'})
fig.update_layout(height=600)
fig.show()

# Song length distribution
fig = px.histogram(songs_df, x='length', nbins=50,
                   title='Song Length Distribution',
                   labels={'length': 'Length (seconds)', 'count': 'Frequency'})
fig.show()

## 4. Defining Customer Churn

Churn is defined as users who visited the 'Cancellation Confirmation' page, as these users completely stop interacting with the platform after this event.

In [14]:
# Identify churned users
churned_users = dataframe_filled.query("page == 'Cancellation Confirmation'")['userId'].unique()

print(f"Total users: {dataframe_filled['userId'].nunique():,}")
print(f"Churned users: {len(churned_users):,}")
print(f"Churn rate: {len(churned_users) / dataframe_filled['userId'].nunique():.1%}")

# Add churn label
dataframe_filled['is_churned'] = dataframe_filled['userId'].isin(churned_users)

Total users: 225
Churned users: 52
Churn rate: 23.1%


In [53]:
# Churn analysis by demographics
churn_by_gender = dataframe_filled.groupby(['userId', 'gender'])['is_churned'].first().groupby('gender').agg(['sum', 'count'])
churn_by_gender['churn_rate'] = churn_by_gender['sum'] / churn_by_gender['count']

print("Churn rate by gender:")
print(churn_by_gender[['churn_rate']])

# Churn by state (top 10 states)
user_state = dataframe_filled.groupby('userId')[['state', 'is_churned']].first()
state_churn = user_state.groupby('state')['is_churned'].agg(['sum', 'count'])
state_churn['churn_rate'] = state_churn['sum'] / state_churn['count']
state_churn = state_churn[state_churn['count'] >= 5].sort_values('churn_rate', ascending=False)

fig = px.bar(x=state_churn.index[:15], y=state_churn['churn_rate'][:15],
             title='Churn Rate by State (Top 15)',
             labels={'x': 'State', 'y': 'Churn Rate'})
fig.show()

# Churn by subscription level
level_churn = dataframe_filled.groupby(['userId', 'level'])['is_churned'].first().groupby('level').agg(['sum', 'count'])
level_churn['churn_rate'] = level_churn['sum'] / level_churn['count']

fig = px.bar(x=level_churn.index, y=level_churn['churn_rate'],
             title='Churn Rate by Subscription Level',
             labels={'x': 'Level', 'y': 'Churn Rate'})
fig.show()

Churn rate by gender:
        churn_rate
gender            
F         0.192308
M         0.264463


## 5. Feature Engineering

Creating features across multiple dimensions:
- Demographic Features
- Activity metrics
- Listening behavior
- Engagement patterns
- Subscription status
- Technical issues
- Temporal patterns
- Session characteristics

In [63]:
# Demographic Features
demographic_features = dataframe_filled.groupby('userId').agg({
    'gender': 'first',
    'state': 'first'
}).fillna('Unknown')

# One-hot encode gender
gender_encoded = pd.get_dummies(demographic_features['gender'], prefix='gender')

# One-hot encode state (top 3 states + 'Other')
state_counts = demographic_features['state'].value_counts()
top_states = state_counts.head(3).index.tolist()
demographic_features['state_grouped'] = demographic_features['state'].apply(
    lambda x: x if x in top_states else 'Other'
)
state_encoded = pd.get_dummies(demographic_features['state_grouped'], prefix='state')

# Combine demographic features
demographic_final = pd.concat([gender_encoded, state_encoded], axis=1)
demographic_final.head()

,gender_F,gender_M,state_CA,state_NY-NJ-PA,state_Other,state_TX
userId,,,,,,
2,True,False,False,False,True,False
3,False,True,False,False,True,False
4,False,True,False,False,True,False
5,False,True,False,False,True,False
6,False,True,False,False,False,True


In [21]:
# Activity Features
user_activity = dataframe_filled.groupby('userId').agg({
    'ts': 'count',                    # total_events
    'sessionId': 'nunique',           # num_sessions  
    'itemInSession': 'sum'            # total_interactions
}).fillna(0)

user_activity.columns = ['total_events', 'num_sessions', 'total_interactions']
user_activity['events_per_session'] = (
    user_activity['total_events'] / user_activity['num_sessions']
).fillna(0)

user_activity.head()

,total_events,num_sessions,total_interactions,events_per_session
userId,,,,
2,926,7,129117,132.285714
3,262,4,12274,65.500000
4,2504,22,380347,113.818182
5,229,6,6459,38.166667
6,3877,24,667436,161.541667


In [22]:
# Listening Behavior Features
songs_df = dataframe_filled[dataframe_filled['song'].notna()]

listening_features = songs_df.groupby('userId').agg({
    'ts': 'count',                    # songs_played
    'length': ['sum', 'mean'],        # total_listening_time, avg_song_length
    'artist': 'nunique',              # unique_artists
    'song': 'nunique',                # unique_songs
    'registration': 'first'           # registration_date
}).fillna(0)

listening_features.columns = ['songs_played', 'total_listening_time', 'avg_song_length', 
                             'unique_artists', 'unique_songs', 'registration_date']

# Calculate days since registration
max_date = dataframe_filled['ts'].max()
listening_features['days_since_registration'] = (
    (max_date - pd.to_datetime(listening_features['registration_date'])).dt.total_seconds() / (24 * 3600)
)

# Daily averages
listening_features['avg_daily_listening_time'] = (
    listening_features['total_listening_time'] / listening_features['days_since_registration']
).fillna(0)

listening_features['avg_daily_songs'] = (
    listening_features['songs_played'] / listening_features['days_since_registration']
).fillna(0)

# Artist diversity
listening_features['artist_diversity'] = (
    listening_features['unique_artists'] / listening_features['songs_played']
).fillna(0)

listening_features = listening_features.drop('registration_date', axis=1)
listening_features.head()

,songs_played,total_listening_time,avg_song_length,unique_artists,unique_songs,days_since_registration,avg_daily_listening_time,avg_daily_songs,artist_diversity
userId,,,,,,,,,
2,755,188687.38342,249.917064,587,713,81.015116,2329.039238,9.319249,0.777483
3,214,54424.74544,254.321240,197,211,114.737095,474.343067,1.865134,0.920561
4,2048,506140.04138,247.138692,1342,1799,65.158021,7767.885441,31.431280,0.655273
5,161,39525.04698,245.497186,154,159,73.418287,538.354252,2.192914,0.956522
6,3159,787236.52359,249.204344,1868,2678,259.476863,3033.937258,12.174496,0.591326


In [24]:
# Engagement Features
thumbs_up = dataframe_filled.query("page == 'Thumbs Up'").groupby('userId').size()
thumbs_down = dataframe_filled.query("page == 'Thumbs Down'").groupby('userId').size()

engagement_features = pd.DataFrame({
    'thumbs_up': thumbs_up,
    'thumbs_down': thumbs_down
}).fillna(0)

engagement_features['total_feedback'] = engagement_features['thumbs_up'] + engagement_features['thumbs_down']
engagement_features['positive_feedback_ratio'] = (
    engagement_features['thumbs_up'] / engagement_features['total_feedback']
).fillna(0.5)

# Other engagement metrics
engagement_features['playlist_adds'] = dataframe_filled[dataframe_filled['page'] == 'Add to Playlist'].groupby('userId').size()
engagement_features['add_friend'] = dataframe_filled[dataframe_filled['page'] == 'Add Friend'].groupby('userId').size()
engagement_features['advert_roll'] = dataframe_filled[dataframe_filled['page'] == 'Roll Advert'].groupby('userId').size()

engagement_features = engagement_features.fillna(0)
engagement_features.head()

,thumbs_up,thumbs_down,total_feedback,positive_feedback_ratio,playlist_adds,add_friend,advert_roll
userId,,,,,,,
2,29.0,6.0,35.0,0.828571,13.0,20.0,0.0
3,14.0,3.0,17.0,0.823529,4.0,1.0,1.0
4,95.0,26.0,121.0,0.785124,59.0,46.0,4.0
5,11.0,0.0,11.0,1.000000,8.0,3.0,11.0
6,165.0,31.0,196.0,0.841837,83.0,41.0,9.0


In [25]:
# Subscription Features
latest_level = dataframe_filled.groupby('userId')['level'].last()
subscription_features = pd.DataFrame({
    'is_paid': (latest_level == 'paid').astype(int)
})

level_changes = dataframe_filled.groupby('userId')['level'].nunique()
subscription_features['subscription_changes'] = (level_changes > 1).astype(int)

subscription_features['downgrades'] = dataframe_filled.query("page == 'Submit Downgrade'").groupby('userId').size()
subscription_features['upgrades'] = dataframe_filled.query("page == 'Submit Upgrade'").groupby('userId').size()

subscription_features = subscription_features.fillna(0)
subscription_features.head()

,is_paid,subscription_changes,downgrades,upgrades
userId,,,,
2,1,0,0.0,0.0
3,1,0,0.0,0.0
4,1,1,0.0,1.0
5,0,0,0.0,0.0
6,1,1,0.0,1.0


In [26]:
# Technical Issues Features
issue_features = pd.DataFrame({
    'error_count': dataframe_filled.query("page == 'Error'").groupby('userId').size(),
    'help_visits': dataframe_filled.query("page == 'Help'").groupby('userId').size(),
    'settings_visits': dataframe_filled.query("page == 'Settings'").groupby('userId').size(),
    'logout_count': dataframe_filled.query("page == 'Logout'").groupby('userId').size()
}).fillna(0)

issue_features['has_issues'] = (
    (issue_features['error_count'] > 0) | 
    (issue_features['help_visits'] > 0)
).astype(int)

issue_features.head()

,error_count,help_visits,settings_visits,logout_count,has_issues
userId,,,,,
2,0.0,6.0,6.0,11.0,1
3,0.0,1.0,0.0,3.0,1
4,4.0,14.0,10.0,24.0,1
5,0.0,2.0,1.0,4.0,1
6,4.0,27.0,17.0,48.0,1


In [27]:
# Temporal Features
last_activity = dataframe_filled.groupby('userId')['ts'].max()
days_since_last_activity = (max_date - last_activity).dt.total_seconds() / (24 * 3600)

user_registration = dataframe_filled.groupby('userId')['registration'].first()
user_registration = pd.to_datetime(user_registration)

days_available = []
for user_id in user_registration.index:
    reg_date = user_registration[user_id]
    days_available.append((max_date.date() - reg_date.date()).days + 1)

days_used = dataframe_filled.groupby('userId')['ts'].apply(lambda x: x.dt.date.nunique())

temporal_features = pd.DataFrame({
    'days_since_last_activity': days_since_last_activity,
    'days_used_in_period': days_used,
    'days_available_in_period': days_available,
    'usage_frequency': days_used / pd.Series(days_available, index=days_used.index)
}, index=user_registration.index).fillna(0)

temporal_features.head()

,days_since_last_activity,days_used_in_period,days_available_in_period,usage_frequency
userId,,,,
2,11.111944,9,82,0.109756
3,33.841817,5,116,0.043103
4,2.360278,26,67,0.388060
5,4.470787,9,75,0.120000
6,3.099190,28,261,0.107280


In [28]:
# Session Pattern Features
session_lengths = dataframe_filled.groupby(['userId', 'sessionId'])['itemInSession'].max()
session_length_stats = session_lengths.groupby('userId').agg(['mean', 'std', 'max']).fillna(0)
session_length_stats.columns = ['avg_session_length', 'session_length_std', 'max_session_length']

session_durations = dataframe_filled.groupby(['userId', 'sessionId'])['ts'].apply(
    lambda x: (x.max() - x.min()).total_seconds() / 60
)
session_duration_stats = session_durations.groupby('userId').agg(['mean', 'std', 'max']).fillna(0)
session_duration_stats.columns = ['avg_session_duration_mins', 'session_duration_std_mins', 'max_session_duration_mins']

session_features = pd.concat([session_length_stats, session_duration_stats], axis=1)
session_features['session_consistency'] = 1 / (1 + session_features['session_length_std'])

session_features.head()

,avg_session_length,session_length_std,max_session_length,avg_session_duration_mins,session_duration_std_mins,max_session_duration_mins,session_consistency
userId,,,,,,,
2,131.285714,150.921298,443,445.116667,516.589537,1506.016667,0.006582
3,64.500000,50.494224,138,247.425000,203.812271,546.016667,0.019420
4,112.818182,150.901619,522,379.487879,526.184277,1836.600000,0.006583
5,37.166667,29.687820,74,24765.052778,38141.830232,74038.766667,0.032586
6,160.541667,176.000489,567,548.161111,601.268049,1913.416667,0.005650


In [64]:
# Initialize feature dataframe
all_users = dataframe_filled['userId'].dropna().unique()
final_features = pd.DataFrame(index=all_users)

# List of all feature DataFrames
feature_sets = [
    demographic_final,       # Demographic features
    user_activity,           # Activity features
    listening_features,      # Listening features  
    engagement_features,     # Engagement features
    subscription_features,   # Subscription features
    issue_features,          # Problem/issue features
    temporal_features,       # Temporal features
    session_features         # Session features
]

# Merge all feature sets
for i, feature_set in enumerate(feature_sets):
    final_features = final_features.join(feature_set, how='left')

# Fill any remaining NaNs with 0
final_features = final_features.fillna(0)

# Add target variable
final_features['is_churned'] = final_features.index.isin(churned_users).astype(int)

print(f"Final feature matrix shape: {final_features.shape}")
print(f"Features created: {len(final_features.columns)-1}")
print(f"Churn rate in final dataset: {final_features['is_churned'].mean():.1%}")

Final feature matrix shape: (225, 47)
Features created: 46
Churn rate in final dataset: 23.1%


## 6. Feature Analysis

In [65]:
# Find features most correlated with churn
feature_cols = [col for col in final_features.columns if col != 'is_churned']
correlations = final_features[feature_cols + ['is_churned']].corr()['is_churned'].abs().sort_values(ascending=False)

print("Top 10 features correlated with churn:")
for feature, corr in correlations.head(11).items():
    if feature != 'is_churned':
        print(f"{feature}: {corr:.3f}")

Top 10 features correlated with churn:
days_since_last_activity: 0.693
positive_feedback_ratio: 0.237
days_used_in_period: 0.198
error_count: 0.191
usage_frequency: 0.184
add_friend: 0.181
thumbs_up: 0.168
unique_artists: 0.162
unique_songs: 0.159
total_feedback: 0.157


### Key Observations:
- **Temporal patterns** (usage frequency, days since last activity) are the strongest predictors of churn
- **Technical issues** (errors, help page visits) are major churn drivers
- **Content diversity** (unique artists/songs) indicates user engagement
- **Positive feedback ratio** shows satisfied users are less likely to churn

## 7. Model Training and Evaluation

In [66]:
# Prepare data for modeling
X = final_features.drop('is_churned', axis=1)
y = final_features['is_churned']

print(f"Training features: {X.shape[1]}")
print(f"Training samples: {len(X)}")
print(f"Class distribution:")
print(y.value_counts())
print(f"Churn rate: {y.mean():.1%}")

Training features: 46
Training samples: 225
Class distribution:
is_churned
0    173
1     52
Name: count, dtype: int64
Churn rate: 23.1%


In [67]:
# Define models and parameter grids
class_counts = y.value_counts()
scale_pos_weight = class_counts[0] / class_counts[1]

def create_models(seed=42):
    """Create model pipelines for evaluation."""
    models = {
        'logistic_regression': Pipeline([
            ('scaler', StandardScaler()),
            ('feature_selection', SelectKBest(f_classif, k=25)),
            ('model', LogisticRegression(
                random_state=seed,
                max_iter=5000,
                solver='liblinear'
            ))
        ]),
        
        'random_forest': Pipeline([
            ('scaler', RobustScaler()),
            ('model', RandomForestClassifier(
                random_state=seed,
                n_jobs=-1
            ))
        ]),
        
        'gradient_boosting': Pipeline([
            ('scaler', StandardScaler()),
            ('model', GradientBoostingClassifier(
                random_state=seed
            ))
        ]),

        'xgboost': Pipeline([
            ('scaler', StandardScaler()),
            ('model', XGBClassifier(
                random_state=seed,
                n_jobs=-1,
                verbosity=0
            ))
        ])
    }
    return models

def get_param_grids():
    """Define hyperparameter grids for each model."""
    param_grids = {
        'logistic_regression': {
            'feature_selection__k': [20, 25, 30, 35],
            'model__C': [0.1, 1, 10, 100],
            'model__penalty': ['l1', 'l2'],
            'model__class_weight': [
                'balanced',
                {0: 1, 1: 2},
                {0: 1, 1: 3},
                {0: 1, 1: 4}
            ]
        },
        
        'random_forest': {
            'model__n_estimators': [100, 200, 300],
            'model__max_depth': [10, 15, 20, None],
            'model__min_samples_split': [2, 5, 10],
            'model__min_samples_leaf': [1, 2, 4],
            'model__class_weight': [
                'balanced',
                {0: 1, 1: 2},
                {0: 1, 1: 3}
            ]
        },
        
        'gradient_boosting': {
            'model__n_estimators': [100, 200, 300],
            'model__learning_rate': [0.05, 0.1, 0.2],
            'model__max_depth': [3, 5, 7],
            'model__subsample': [0.8, 0.9, 1.0]
        },

        'xgboost': {
            'model__n_estimators': [100, 200],
            'model__max_depth': [3, 4, 5],
            'model__learning_rate': [0.1, 0.2],
            'model__scale_pos_weight': [2, 3, 4]
        }
    }
    return param_grids

In [68]:
# Hyperparameter tuning with cross-validation
models = create_models()
param_grids = get_param_grids()
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

best_models = {}
results_summary = []

for model_name, model in models.items():
    print(f"\nTuning {model_name}...")
    
    grid_search = GridSearchCV(
        model,
        param_grids[model_name],
        cv=cv,
        scoring='f1',
        n_jobs=-1,
        verbose=0
    )
    
    grid_search.fit(X, y)
    
    # Store best model and results
    best_models[model_name] = grid_search.best_estimator_
    
    # Evaluate best model
    cv_scores = cross_val_score(grid_search.best_estimator_, X, y, cv=cv, scoring='f1')
    
    results_summary.append({
        'model': model_name,
        'best_params': grid_search.best_params_,
        'best_score': grid_search.best_score_,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std()
    })
    
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"F1 Score: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")


Tuning logistic_regression...
Best parameters: {'feature_selection__k': 25, 'model__C': 1, 'model__class_weight': {0: 1, 1: 2}, 'model__penalty': 'l1'}
F1 Score: 0.8282 (+/- 0.0982)

Tuning random_forest...
Best parameters: {'model__class_weight': 'balanced', 'model__max_depth': 10, 'model__min_samples_leaf': 4, 'model__min_samples_split': 2, 'model__n_estimators': 100}
F1 Score: 0.6499 (+/- 0.1162)

Tuning gradient_boosting...
Best parameters: {'model__learning_rate': 0.05, 'model__max_depth': 7, 'model__n_estimators': 300, 'model__subsample': 0.9}
F1 Score: 0.7368 (+/- 0.1420)

Tuning xgboost...
Best parameters: {'model__learning_rate': 0.1, 'model__max_depth': 5, 'model__n_estimators': 100, 'model__scale_pos_weight': 2}
F1 Score: 0.7845 (+/- 0.1079)


In [69]:
# Compare model performance
results_df = pd.DataFrame(results_summary)
results_df = results_df.sort_values('cv_mean', ascending=False)

print("\nModel Performance Summary:")
print(results_df[['model', 'cv_mean', 'cv_std']].to_string(index=False))

# Select best model
best_model_name = results_df.iloc[0]['model']
best_model = best_models[best_model_name]
print(f"\nBest performing model: {best_model_name}")


Model Performance Summary:
              model  cv_mean   cv_std
logistic_regression 0.828175 0.098187
            xgboost 0.784453 0.107866
  gradient_boosting 0.736761 0.141960
      random_forest 0.649910 0.116237

Best performing model: logistic_regression


In [70]:
# Get predictions using cross-validation
y_pred = cross_val_predict(best_model, X, y, cv=cv)

print(f"\nBest Model: {best_model_name}")
print("\nClassification Report:")
print(classification_report(y, y_pred))

print("\nConfusion Matrix:")
cm = confusion_matrix(y, y_pred)
print(cm)


Best Model: logistic_regression

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       173
           1       0.83      0.83      0.83        52

    accuracy                           0.92       225
   macro avg       0.89      0.89      0.89       225
weighted avg       0.92      0.92      0.92       225


Confusion Matrix:
[[164   9]
 [  9  43]]


In [71]:
# Feature importance analysis (if applicable)
if hasattr(best_model.named_steps['model'], 'feature_importances_'):
    # For tree-based models
    feature_importance = best_model.named_steps['model'].feature_importances_
    feature_names = X.columns
    
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': feature_importance
    }).sort_values('importance', ascending=False)
    
    print("\nTop 15 Most Important Features:")
    print(importance_df.head(15).to_string(index=False))
    
elif hasattr(best_model.named_steps['model'], 'coef_'):
    # For linear models with feature selection
    selector = best_model.named_steps['feature_selection']
    selected_features = X.columns[selector.get_support()]
    coefficients = best_model.named_steps['model'].coef_[0]
    
    importance_df = pd.DataFrame({
        'feature': selected_features,
        'coefficient': coefficients,
        'abs_coefficient': np.abs(coefficients)
    }).sort_values('abs_coefficient', ascending=False)
    
    print("\nTop 15 Most Important Features (by absolute coefficient):")
    print(importance_df.head(15)[['feature', 'coefficient']].to_string(index=False))


Top 15 Most Important Features (by absolute coefficient):
                 feature  coefficient
days_since_last_activity     3.022104
     session_consistency    -1.362850
        artist_diversity    -1.285926
      session_length_std    -0.720758
              add_friend    -0.460704
 positive_feedback_ratio    -0.447760
avg_daily_listening_time    -0.171787
            unique_songs     0.000000
             help_visits     0.000000
      max_session_length     0.000000
      total_interactions     0.000000
         usage_frequency     0.000000
     days_used_in_period     0.000000
            songs_played     0.000000
            logout_count     0.000000


### Model Robustness Testing

To ensure our model generalizes well, we'll test it across multiple random seeds.

In [72]:
# Test model stability across different random seeds
random_seeds = [42, 123, 456, 789, 101112]
seed_results = []

for seed in random_seeds:
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    
    # Create model with new seed
    if best_model_name == 'logistic_regression':
        model = create_models(seed)[best_model_name]
        best_params = results_df[results_df['model'] == best_model_name].iloc[0]['best_params']
        
        # Apply best parameters
        for param, value in best_params.items():
            step, param_name = param.split('__')
            setattr(model.named_steps[step], param_name, value)
    else:
        model = best_models[best_model_name]
    
    scores = cross_val_score(model, X, y, cv=cv, scoring='f1')
    seed_results.append({
        'seed': seed,
        'mean_f1': scores.mean(),
        'std_f1': scores.std()
    })

seed_df = pd.DataFrame(seed_results)
print("Model stability across random seeds:")
print(seed_df.to_string(index=False))
print(f"Overall mean F1: {seed_df['mean_f1'].mean():.4f}")
print(f"Stability (std of means): {seed_df['mean_f1'].std():.4f}")

Model stability across random seeds:
  seed  mean_f1   std_f1
    42 0.828175 0.098187
   123 0.781445 0.048381
   456 0.744710 0.091847
   789 0.817835 0.049177
101112 0.805931 0.100830

Overall mean F1: 0.7956
Stability (std of means): 0.0334


Exception ignored in: <function ResourceTracker.__del__ at 0x107e545e0>
Traceback (most recent call last):
  File "/Users/yasserjaber/miniforge3/envs/customer_churn/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/Users/yasserjaber/miniforge3/envs/customer_churn/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/Users/yasserjaber/miniforge3/envs/customer_churn/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x1061585e0>
Traceback (most recent call last):
  File "/Users/yasserjaber/miniforge3/envs/customer_churn/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/Users/yasserjaber/miniforge3/envs/customer_churn/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/Users/yasserjaber/miniforge3/envs/customer_churn/lib/python3.13/multipr

## 8. Final Model and Conclusions

The model evaluation shows strong performance with good stability across different random seeds, indicating that our approach generalizes well.

### Key Findings:
1. **Best Model**: Logistic Regression achieved the highest F1 score with good interpretability
2. **Important Features**: Temporal patterns (days since last activity, usage frequency) are the strongest predictors
3. **Technical Issues Impact**: Error counts and help page visits strongly correlate with churn
4. **Engagement Metrics**: Positive feedback ratio and content diversity indicate user satisfaction
5. **Model Stability**: Low variance across random seeds confirms robust performance

### Business Insights:
- Focus retention efforts on users showing decreased activity patterns
- Proactively address technical issues to prevent churn
- Encourage user engagement through personalized content recommendations
- Monitor users with declining usage frequency for early intervention

### Next Steps:
1. Implement production pipeline with FastAPI
2. Set up MLflow for experiment tracking
3. Build monitoring system for data and concept drift
4. Create automated retraining workflows
5. Develop business dashboards for stakeholder visibility

In [75]:
import joblib 

# save model with joblib 
filename = 'lg_churn.pkl'
joblib.dump(best_model, filename)

['lg_churn.pkl']